## Coletando dados
##### Coletando dados salvos estaticamente na pasta `./data`

In [100]:
import pandas as pd

df = pd.read_csv("./data/data.csv",low_memory=False)


## Convertendo valor de renda per capita para numérico
##### Extraindo os valores de `Renda Per Capita`, transformando de categorico para numerico, atraves da media entre o menor e o maior valor. 

In [101]:
import re 
import numpy as np

def convert_renda_per_capita(value):
    if isinstance(value, str):
        match = re.match(r"(\d+,\d+|\d+) Sm < Rfp <= (\d+,\d+|\d+) Sm", value)
        if match:
            lower_bound = float(match.group(1).replace(",", "."))
            upper_bound = float(match.group(2).replace(",", "."))
            return (lower_bound + upper_bound) / 2
    return np.nan

df["Renda per Capita"] = df["Renda per Capita"].apply(convert_renda_per_capita)

## Pré-processando valores nulos 
##### Processando valores nulos, para valores categoricos atribuir categoria `Desconhecido`, para variaveis numericas, tirando `Renda Per Capita`, atribuimos a media dos valores daquela coluna. Para fins de desenvolvimento, salvamos os dados processados na pasta `./data/processed_data.csv`


In [102]:
def pre_process(df) -> pd.DataFrame:
    df = df.copy()
    for coluna in df.columns:
        if df[coluna].dtype in ["float64", "int64"]:  
            if coluna == "Renda per Capita":
                df = df[df["Renda per Capita"].notnull() & (df["Renda per Capita"] != "")]
            df[coluna] = df[coluna].fillna(df[coluna].mean())  
        elif df[coluna].dtype == "object":
                df[coluna] = df[coluna].apply(
                    lambda x: "Desconhecido" if coluna != "Sexo" and (pd.isna(x) or len(str(x)) < 3) else x
                )
    return df

df = pre_process(df)
df.to_csv('./data/processed_data.csv', index=False)

## Tratando outliers
##### Através do modelo `isolation-florest`, conseguimos extrair os padrões dos dados do nosso dataset, e através desse padrao conseguimos dizer se um certo valor está dentro do padrão ou não, atribuindo um score que é analisado pelo modelo, e de acordo com a contaminação estabelecida e o padrão dos demais scores, deleta ou mantem o valor em questão. Para fins de desenvolvimento, salvamos os dados tratados na pasta `./data/cleaned_data.csv`

In [103]:
from sklearn.ensemble import IsolationForest

def remove_outliers(df):
    isolation_forest = IsolationForest(contamination=0.05)  

    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df_numeric = df[numeric_columns]
    
    outliers = isolation_forest.fit_predict(df_numeric)

    df_cleaned = df[outliers == 1]
    
    return df_cleaned

df = remove_outliers(df)
df.to_csv('./data/cleaned_data.csv',index=False)

## Definindo features e o target
##### Definindo de forma estatica as colunas que serão as features usadas nos modelos de regressão, bem como a coluna alvo.

In [104]:
categorical_columns = [
        "Tipo do Curso",
]
numeric_columns = [
]

target_column = "Renda per Capita"

## Transformando os dados
##### Nessa etapa, fazemos uma mistura de técnicas de transformação de dados, como `one-hot-encoding` para transformar as variaveis categoricas em variaveis numéricas.

In [105]:
from sklearn.preprocessing import OneHotEncoder

def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    X_categorical = df[categorical_columns]
    X_additional = df[numeric_columns]
    y = df[target_column]
    
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_encoded = encoder.fit_transform(X_categorical)
    
    X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_columns), index=df.index)
    
    df_encoded = pd.concat([X_encoded_df, X_additional, y], axis=1)
    
    return df_encoded


df = transform_data(df)
df.to_csv('./data/encoded_data.csv',index=False)



## Separando dataset
#### Nessa etapa, estamos separando os nossos dados, divindo entre treino e teste. Nesse caso foi utilizado 70% dos dados para treino e 30% para teste. A biblioteca utilizada foi `sklearn.model_selection` com a funcao `train_test_split`

In [106]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=[target_column])  
y = df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Treinamento com modelos de regressão
##### Essa etapa consiste em fazer o split do nosso dataset, em dados de treino e de teste, e realizar o treinamento dos nossos dados com o modelo de `linear-regression` e `decision-tree-regression`, ambos da biblioteca `sklearn`

In [107]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred_linear = linear_model.predict(X_test)

tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

y_pred_tree = tree_model.predict(X_test)


## Aplicando testes
##### Nessa etapa, iremos aplicar os testes, e ver como os nossos dados se comportaram diante os dois modelos utilizados, e analisar caracteristicas como viés e variância dos nossos dados, bem como o resultado dos nossos modelos de regressão

In [108]:
from sklearn.metrics import mean_squared_error
from mlxtend.evaluate import bias_variance_decomp
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_linear_test = rmse(y_test, y_pred_linear)
rmse_linear_train = rmse(y_train, linear_model.predict(X_train))

rmse_tree_test = rmse(y_test, y_pred_tree)
rmse_tree_train = rmse(y_train, tree_model.predict(X_train))

print(f"Linear Regression - Training RMSE: {rmse_linear_train}")
print(f"Linear Regression - Test RMSE: {rmse_linear_test}")
print(f"Decision Tree - Training RMSE: {rmse_tree_train}")
print(f"Decision Tree - Test RMSE: {rmse_tree_test}")

linear_bias, linear_variance, linear_error = bias_variance_decomp(linear_model, X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy(),random_seed=123, loss='mse')

tree_bias, tree_variance, tree_error = bias_variance_decomp(tree_model, X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy(),random_seed=123, loss='mse')

print(f"Linear Regression - Bias: {linear_bias}")
print(f"Linear Regression - Variance: {linear_variance}")
print(f"Linear Regression - Error: {linear_error}")

print(f"Decision Tree - Bias: {tree_bias}")
print(f"Decision Tree - Variance: {tree_variance}")
print(f"Decision Tree - Error: {tree_error}")


Linear Regression - Training RMSE: 0.5960251725321846
Linear Regression - Test RMSE: 0.6012181999927876
Decision Tree - Training RMSE: 0.5960251725321846
Decision Tree - Test RMSE: 0.6012181999927876
Linear Regression - Bias: 0.36187975146463325
Linear Regression - Variance: 0.3614334055522992
Linear Regression - Error: 0.00044634591233400826
Decision Tree - Bias: 0.36187975146463325
Decision Tree - Variance: 0.36143340555229914
Decision Tree - Error: 0.0004463459123340106
